In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import sys
sys.path.append('./scripts/')

In [3]:
from update_vars import (analysis_date, AM_PEAK, PM_PEAK, EXPORT_PATH, GCS_FILE_PATH, PROJECT_CRS,
SEGMENT_BUFFER_METERS, AM_PEAK, PM_PEAK, HQ_TRANSIT_THRESHOLD, MS_TRANSIT_THRESHOLD, HALF_MILE_BUFFER_METERS,
                        MPO_DATA_PATH)

In [4]:
from calitp_data_analysis.tables import tbls

In [5]:
from calitp_data_analysis import get_fs
fs = get_fs()
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
gcsgp = GCSGeoPandas()

In [6]:
import assemble_hqta_points

In [20]:
mpos = [x.split('/')[-1].split('.')[0] for x in fs.ls(MPO_DATA_PATH) if x.split('/')[-1] != 'mpo_input']

In [16]:
fs.ls(MPO_DATA_PATH)[0].split('/')

['calitp-analytics-data',
 'data-analyses',
 'high_quality_transit_areas',
 'mpo_input']

In [17]:
MPO_DATA_PATH

'gs://calitp-analytics-data/data-analyses/high_quality_transit_areas/mpo_input/'

In [18]:
mpos

['mtc', 'sacog', 'sandag', 'scag']

In [15]:
def read_standardize_mpo_input(mpo_name: str, mpo_data_path = MPO_DATA_PATH, gcsgp = gcsgp) -> gpd.GeoDataFrame:
    """
    Read in mpo-provided planned major transit stops and enforce schema.
    """
    mpo_gdf = gcsgp.read_file(f'{MPO_DATA_PATH}{mpo_name}.geojson')
    required_cols = ['mpo', 'hqta_type', 'plan_name']
    optional_cols = ['stop_id', 'avg_trips_per_peak_hr', 'agency_primary']
    all_cols = required_cols + optional_cols + ['geometry']
    assert set(required_cols).issubset(mpo_gdf.columns)
    filter_cols = [col for col in all_cols if col in mpo_gdf.columns]
    mpo_gdf = mpo_gdf[filter_cols]
    return mpo_gdf

In [16]:
mpo_planned_mts = pd.concat([read_standardize_mpo_input(mpo_name) for mpo_name in mpos])

In [17]:
mpo_planned_mts

,mpo,hqta_type,plan_name,agency_primary,geometry,stop_id,avg_trips_per_peak_hr
0,MTC,major_stop_brt,Plan Bay Area 2050 (2021),AC Transit,POINT (-122.29408 37.87535),NaN,NaN
1,MTC,major_stop_brt,Plan Bay Area 2050 (2021),SamTrans,POINT (-122.35600 37.57840),NaN,NaN
2,MTC,major_stop_brt,Plan Bay Area 2050 (2021),SamTrans,POINT (-122.34807 37.57595),NaN,NaN
3,MTC,major_stop_brt,Plan Bay Area 2050 (2021),SamTrans,POINT (-122.34752 37.57564),NaN,NaN
4,MTC,major_stop_brt,Plan Bay Area 2050 (2021),SamTrans,POINT (-122.34291 37.57451),NaN,NaN
...,...,...,...,...,...,...,...
1345,SCAG,major_stop_rail,2024 RTP,NaN,POINT (-118.16458 33.91167),NaN,NaN
1346,SCAG,major_stop_rail,2024 RTP,NaN,POINT (-118.22498 33.98541),NaN,NaN
1347,SCAG,major_stop_rail,2024 RTP,NaN,POINT (-118.16146 33.90424),NaN,NaN
1348,SCAG,major_stop_rail,2024 RTP,NaN,POINT (-118.08148 33.85992),NaN,NaN
